In [1]:
import networkx as nx
import scipy as sp
import numpy as np
import random
import copy
from numpy import linalg

%matplotlib inline

In [2]:
political=nx.read_gml("polblogs.gml",label='id')

#make the graph directed with no parallels
political = nx.DiGraph(political)

#returns a set of nodes of the biggest connected component
largest = max(nx.connected_components(political.to_undirected()), key=len)

# creates the graph of the biggest connected component including direction
CC_max = political.subgraph(largest)


In [3]:
def calculate_avg_score(G, node_id):
    sum_of_score = 0
    number_of_neighbors = 0
    G=G.to_undirected()
    for neighbor_id in G[node_id]:
        sum_of_score += G.nodes[neighbor_id]['value']
        number_of_neighbors += 1
        
    return sum_of_score / number_of_neighbors

In [4]:
def propagate(G):
    next_scores = {}
    for node_id in G.nodes():
        if G.nodes[node_id]['value'] is 1 or G.nodes[node_id]['value'] is 0:
            #if the node value is known it wont be changed
            next_scores[node_id] = G.nodes[node_id]['value']
        else:
            next_scores[node_id] = calculate_avg_score(G, node_id)
    return next_scores

In [ ]:
#Returns a list with the value of the nodes
original_values = dict(CC_max.nodes(data='value', default=1))

#get the b% of values
percentage=0.1

#a small value that stops the propagation
convergence_factor = 0.00001

#list that holds the values of every experiment
precision_list = []

for i in range(9):
    
    numbers_of_items_kept_from_percentage = round(len(a)*percentage)
    sum_of_precision = 0
    
    print("Now computing nodes for:",percentage)
    print("========================")
    
    for i in range(10):
        
        # shuffle the dictionary of values to get the random nodes
        a1 = list(original_values.items())
        np.random.shuffle(a1)
        a = dict(a1)

        #keep the first percentage b% of the nodes
        nodes_kept = list(a.items())[:numbers_of_items_kept_from_percentage]


        #holds the b% nodes with their original values, the rest of the nodes are initialized to 0.5
        final_dictionary = a.copy()

        #initialize not known node values
        for key in dict(nodes_kept).keys():
            final_dictionary[key] = 0.5

        for key in final_dictionary.keys():
            attrs = {key: {'value': final_dictionary[key]}}
            nx.set_node_attributes(CC_max,attrs)


        graph_to_be_propagated = CC_max.copy()


        #for the first convergence comparison

        previous_values = nx.get_node_attributes(CC_max,'value')
        previous_values_for_comparison = list(previous_values.values())
        norm_of_previous_values = linalg.norm(previous_values_for_comparison,1)

        #propagate until convergence

        while True:

            next_values = propagate(graph_to_be_propagated)
            list_of_next_values = list(next_values.values())
            norm_of_next_values = linalg.norm(list_of_next_values,1)


            norm_difference = abs(norm_of_next_values - norm_of_previous_values)

            #print(norm_difference)

            if norm_difference < convergence_factor:
                break

            #update the new values of the graph
            for key in next_values.keys():
                attrs = {key: {'value': next_values[key]}}
                nx.set_node_attributes(graph_to_be_propagated,attrs)

            #update the previous values
            previous_values = copy.deepcopy(next_values)

            #update the previous values norm for the next comparison
            list_of_previous_values = list(previous_values.values())
            norm_of_previous_values = linalg.norm(list_of_previous_values,1)


        #predict the node values    
        nodes = dict(graph_to_be_propagated.nodes(data='value', default=1))


        #update the new values of the graph
        for key in nodes.keys():
            if nodes[key] > 0.5:
                attrs = {key: {'value': 1}}
            else:
                attrs = {key: {'value': 0}}
            nx.set_node_attributes(graph_to_be_propagated,attrs)


        #Now compare original values with the values_after_propagation
        
        nodes = dict(graph_to_be_propagated.nodes(data='value', default=1))

        shared_items = {k: nodes[k] for k in nodes if k in original_values and nodes[k] == original_values[k]}
        
        
        # accuracy logic
        #
        # TP = no_of_shared_that_are_same - len(b%)
        #
        # FP = no_of_all_nodes - len(b%) - TP
        # FP = no_of_all_nodes - len(b%) - no_of_shared_that_are_same + len(b%)
        # FP = no_of_all_nodes - no_of_shared_that_are_same
        
        
        true_positive = len(shared_items) - numbers_of_items_kept_from_percentage
        
        false_positive = len(original_values) - len(shared_items)
        
        precision = true_positive / (true_positive + false_positive)
        
        sum_of_precision += precision
        
    precision_results = sum_of_precision / 10
    precision_list.append(precision_results)
    
    #update percentage
    percentage += 0.1

Now computing nodes for: 0.1
